In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [13]:
FILE_PATH = "eurpln_d.csv"
BATCH_SIZE = 4
TRAIN_TEST_RATIO = 0.8
NUM_EPOCHS = 40
LR_RATE = 0.0001
IN_SIZE = 3

In [14]:
data = pd.read_csv(FILE_PATH)
data = data.drop(["Data","Otwarcie","Najwyzszy","Najnizszy"], axis=1)
data.dropna(axis=0,inplace=True)
data = torch.Tensor(data.to_numpy())

data_min = torch.min(data, dim=0, keepdim=True).values
data_max = torch.max(data, dim=0, keepdim=True).values

In [15]:
class MyDataset(Dataset):
    def __init__(self, data, shift_size, data_min, data_max):
        self.data = data
        self.labels = self.data
        self.shift_size = shift_size
        self.data = (self.data - data_min)/(data_max-data_min)
    
    def __len__(self):
        return self.data.shape[0] - self.shift_size

    def __getitem__(self, idx):
        idx += self.shift_size
        x = self.data[(idx-self.shift_size):idx].t()
        y = self.labels[idx].unsqueeze(1)
        
        return x, y

In [16]:
idx = int(TRAIN_TEST_RATIO*len(data))
train_data = data[:idx]
test_data = data[idx:]

In [17]:
dataset_train = MyDataset(train_data, IN_SIZE, data_min, data_max)
dataset_test = MyDataset(test_data, IN_SIZE, data_min, data_max)
trainloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=False)
testloader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

In [18]:
x, y = next(iter(trainloader))
y.shape

torch.Size([4, 1, 1])

In [19]:
class NeuralNetwork(nn.Module):
    def __init__ (self,input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size,16)
        self.fc2 = nn.Linear(16,8)
        self.fc3 = nn.Linear(8,1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.relu(self.fc3(x))
        return x
    
    

In [20]:
model = NeuralNetwork(input_size=IN_SIZE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR_RATE)

In [21]:
#learning the model
for epoch in range(NUM_EPOCHS):
    print("epoch: {}".format(epoch))
    train_loss = 0.0
    for x, y in trainloader:
        #zero the parameters gradient
        optimizer.zero_grad()
        
        #forward, backward, optimize
        output = model(x)
        loss = criterion(output, y)
        train_loss += loss
        loss.backward()
        optimizer.step()
    print("train_loss: {}".format(train_loss / len(trainloader)))
    
    
    test_loss = 0.0
    model.eval()
    for x, y in testloader:
        output = model(x)
        loss = criterion(output, y)
        test_loss += loss
    print("test_loss: {}".format(test_loss / len(testloader)))
    model.train()
        

epoch: 0
train_loss: 14.565702438354492
test_loss: 13.751509666442871
epoch: 1
train_loss: 12.961514472961426
test_loss: 11.860633850097656
epoch: 2
train_loss: 11.082265853881836
test_loss: 9.61493968963623
epoch: 3
train_loss: 8.872008323669434
test_loss: 7.064929485321045
epoch: 4
train_loss: 6.456127166748047
test_loss: 4.490236282348633
epoch: 5
train_loss: 4.1398468017578125
test_loss: 2.3239939212799072
epoch: 6
train_loss: 2.292426347732544
test_loss: 0.9247596263885498
epoch: 7
train_loss: 1.13287353515625
test_loss: 0.31722190976142883
epoch: 8
train_loss: 0.5865617394447327
test_loss: 0.19273848831653595
epoch: 9
train_loss: 0.39235085248947144
test_loss: 0.21879900991916656
epoch: 10
train_loss: 0.33201754093170166
test_loss: 0.24674877524375916
epoch: 11
train_loss: 0.30855241417884827
test_loss: 0.2538841962814331
epoch: 12
train_loss: 0.29376253485679626
test_loss: 0.24870407581329346
epoch: 13
train_loss: 0.2810157239437103
test_loss: 0.2385445386171341
epoch: 14
train_

In [11]:
x,y = next(iter(testloader))
print(x)
print(y)
print(model(x))

tensor([[[0.6157, 0.5877, 0.6540]],

        [[0.5877, 0.6540, 0.6526]],

        [[0.6540, 0.6526, 0.7049]],

        [[0.6526, 0.7049, 0.6842]]])
tensor([[[4.3072]],

        [[4.3210]],

        [[4.3155]],

        [[4.3202]]])
tensor([[[4.2970]],

        [[4.3082]],

        [[4.3371]],

        [[4.3626]]], grad_fn=<ReluBackward0>)


# making predictions
predictions = []
net.eval()
for i, data in enumerate(test):
    
    inputs, labels = data[1:4], data[0]

    predicted = net(inputs)
    predictions.append(predicted)

In [ ]:
print(test_min_max[1][0][0])
print(predictions[:10])
test_min_max[0][0][0]

In [ ]:
#najpierw "odskaluje" dane
predictions = torch.Tensor(predictions)
print(predictions[:10])
predictions = predictions*(test_min_max[1][0][0]-test_min_max[0][0][0])+test_min_max[0][0][0]
#tutaj pasuje sie zastanowic czy nie ma bledu metodologicznego
#bo mam watpliwosci czy uzywanie statystyk zbioru testowego dla predykcji jest uzasadnione
test = test*(test_min_max[1]-test_min_max[0])+test_min_max[0]

plt.plot(np.array(predictions))
plt.plot(test[:,0])
plt.show()
predictions[:10]